In [1]:
import os

In [2]:
%pwd

'd:\\ML Projects\\End-to-End-Red-Wine-Quality-Prediction-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML Projects\\End-to-End-Red-Wine-Quality-Prediction-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    target_column : str

In [16]:
from Red_Wine_Quality_Prediction.constants import *
from Red_Wine_Quality_Prediction.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
            
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import joblib

In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred) * 100
        metrics = classification_report(actual, pred)
        return metrics, accuracy

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        pred = model.predict(test_x)

        metrics, accuracy = self.eval_metrics(test_y, pred)

        scores = {
            "accuracy": accuracy,
            "classification_report": metrics
        }

        save_json(path=Path(self.config.metric_file_name), data=scores)

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-07-03 16:51:56,581: INFO: common : yaml file: config\config.yaml loaded successfully]
[2024-07-03 16:51:56,583: INFO: common : yaml file: params.yaml loaded successfully]
[2024-07-03 16:51:56,586: INFO: common : yaml file: schema.yaml loaded successfully]
[2024-07-03 16:51:56,587: INFO: common : created directory at: artifacts]
[2024-07-03 16:51:56,589: INFO: common : created directory at: artifacts/model_evaluation]
[2024-07-03 16:51:57,364: INFO: common : json file saved at: artifacts\model_evaluation\metrics.json]
